In [ ]:
# Philip Walsh
# philipwalsh.ds@gmail.com
# 2019-12-20
# Starter notebook can be used as a jumping off point to the challenge
# Kaggle - House Prices: Advanced Regression Techniques
# https://www.kaggle.com/c/house-prices-advanced-regression-techniques


In [ ]:
# complete_train_data  : training.csv (provided, contains SalePrice)
# X_train              : 80% of the training data - for fitting the model
# X_hold_out           : 20% of the training data - for evaluating the model on unseen data
# submissions_data     : test.csv (provided, does not contain SalePrice)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

In [ ]:
save_intermediate_work = True

In [ ]:
def best_fit(X, Y):
    # to plot the line of best fit
    # https://stackoverflow.com/questions/22239691/code-for-best-fit-straight-line-of-a-scatter-plot-in-python
    # aziz alto
    xbar = sum(X)/len(X)
    ybar = sum(Y)/len(Y)
    n = len(X) # or len(Y)

    numer = sum([xi*yi for xi,yi in zip(X, Y)]) - n * xbar * ybar
    denum = sum([xi**2 for xi in X]) - n * xbar**2

    b = numer / denum
    a = ybar - b * xbar

    print('best fit line:\ny = {:.2f} + {:.2f}x'.format(a, b))

    return a, b

In [ ]:
##usage
##best fit line:
##y = 0.80 + 0.92x
#
#a, b = best_fit(X, Y)
#
## plot points and fit line
#plt.scatter(X, Y)
#yfit = [a + b * xi for xi in X]
#plt.plot(X, yfit)

In [ ]:
complete_train_data = pd.read_csv('excluded/train.csv', low_memory=False)

In [ ]:
complete_train_data.shape

In [ ]:
complete_train_data.describe()

In [ ]:
# SalePrice is what we are being asked to predict
# so lets havea quick look 
plt.hist(complete_train_data['SalePrice'])

In [ ]:
plt.hist(np.log(complete_train_data['SalePrice']))

In [ ]:
complete_train_data['LotFrontage'].fillna(complete_train_data['LotFrontage'].mean(), inplace=True)

In [ ]:
#scatter plot of LotFrontage vs SalePrice
# add in line of best fit as well
X = complete_train_data['LotFrontage']
Y = complete_train_data['SalePrice']
a, b = best_fit(X, Y)

# plot points and fit line
plt.scatter(X, Y)
plt.xlabel('LotFrontage')
plt.ylabel('SalePrice')
yfit = [a + b * xi for xi in X]
plt.plot(X, yfit, color='red')

In [ ]:
plt.hist(complete_train_data['LotFrontage'])

In [ ]:
X = complete_train_data['GrLivArea']
Y = complete_train_data['SalePrice']
a, b = best_fit(X, Y)
#best fit line:
#y = 0.80 + 0.92x

# plot points and fit line
plt.xlabel('GrLivArea')
plt.ylabel('SalePrice')
plt.scatter(X, Y)
yfit = [a + b * xi for xi in X]
plt.plot(X, yfit, color='red')

In [ ]:
plt.xlabel('GrvLivArea')
plt.boxplot(complete_train_data['GrLivArea'])

In [ ]:
##
## pull in the "test.csv" so we can do some feature engineering and data cleaning
##

In [ ]:
# pull in the submission data, aka the test data
# we will merge the train/test data into one large set, for cleaning purposes
submission_data = pd.read_csv('excluded/test.csv', low_memory=False)

In [ ]:
submission_data.shape

In [ ]:
submission_data.describe()

In [ ]:
##
## steps will be
##  Split up the train data into train/hold_out
##  Tag all 3 sets with a variable that identifies their purpose
##  Combine all 3 sets train+hold_out+submission(aka test)
##

In [ ]:
# set up bins for the stratified split
complete_train_data['living_area_cat'] = pd.cut(
    complete_train_data['GrLivArea'], 
    bins=[0, 500, 1000, 1500, 2000, 2500, np.inf], 
    labels=[1, 2, 3, 4, 5, 6])

In [ ]:
complete_train_data[['SalePrice','GrLivArea','living_area_cat']].head()

In [ ]:
split = StratifiedShuffleSplit(n_splits=1, test_size=.20, random_state=9261774)
for train_index, hold_out_index in split.split(complete_train_data, complete_train_data['living_area_cat']):
    X_train = complete_train_data.iloc[train_index].copy() # this is the training data
    X_hold_out = complete_train_data.iloc[hold_out_index].copy()   # this is the hold out, the protion of the training i will use for testing

In [ ]:
## next we save the SalePrice, remove it from the train/hold_out set

In [ ]:
# set up the label/expected/y variable
y_train = X_train['SalePrice']
y_hold_out = X_hold_out['SalePrice']


In [ ]:
# drop SalePrice from the x vars
X_train.drop('SalePrice', axis=1, inplace=True)
X_hold_out.drop('SalePrice', axis=1, inplace=True)

In [ ]:
# done with the living_area_cat
X_train.drop('living_area_cat', axis=1, inplace=True)
X_hold_out.drop('living_area_cat', axis=1, inplace=True)
complete_train_data.drop('living_area_cat', axis=1, inplace=True)

In [ ]:
# create a new TRAIN field to keep the different data sets from getting mixed up
X_train['TRAIN']=1          # 1 indicates its from the training data
X_hold_out['TRAIN']=0       # 0 indicates its hold-out
submission_data['TRAIN']=-1 #-1 for the submissions data

In [ ]:
X_train.describe()

In [ ]:
X_hold_out.describe()

In [ ]:
submission_data.describe()

In [ ]:
combined=pd.concat([X_train, X_hold_out, submission_data],sort=False)

In [ ]:
combined.describe()

In [ ]:
combined.head()

In [ ]:
combined.tail()

In [ ]:
# look for missing data
#pd.set_option('display.max_rows', 10)
#pd.set_option('display.max_rows', None)
combined[['OverallQual','OverallCond','LotFrontage']].isna().any()

In [ ]:
def CleanData(df_in):
    # lotFrontage was identified as a column with missing data
    # let's do a quick fill
    df_clean = df_in.copy()
    temp_mean = df_clean['LotFrontage'].mean()
    temp_mean
    df_clean['LotFrontage'].fillna(temp_mean, inplace=True)
    
    # multiply the condition and quality to create a new variable
    df_clean['OverallQualCond'] = (df_clean['OverallQual']+1) * (df_clean['OverallCond']+1)
    df_clean.drop('OverallQual',axis=1, inplace=True)
    df_clean.drop('OverallCond',axis=1, inplace=True)
    
    #create a centered LotFrontage
    df_clean['LotFrontageCtr']=df_clean['LotFrontage']-temp_mean
    
    #create a normalized LotFrontage
    df_clean['LotFrontageNorm']=(df_clean['LotFrontage']-df_clean['LotFrontage'].min())/(df_clean['LotFrontage'].max()-df_clean['LotFrontage'].min())
    
    return df_clean

In [ ]:
combined = CleanData(combined)

In [ ]:
combined.head()

In [ ]:
##
## data is clean, features engineered
## time to ripe the sets apart again, and do some modeling

In [ ]:
# train - use to fit models
X_train = combined[combined['TRAIN']==1].copy()
X_train.drop('TRAIN', axis=1, inplace=True)

# sendtofile(excluded_dir, 'X_train.csv', X_train, verbose=True)


# hold_out - use to evaluate model performance
X_hold_out = combined[combined['TRAIN']==0].copy()
X_hold_out.drop('TRAIN', axis=1, inplace=True)

# submission - use to submit the final answer for the kaggle cometition
X_submission = combined[combined['TRAIN']==-1].copy()
X_submission.drop('TRAIN', axis=1, inplace=True)

In [ ]:
X_train.shape

In [ ]:
X_hold_out.shape

In [ ]:
X_train.describe()

In [ ]:
#train_cols = ['LotFrontage','GrLivArea']  #0.5049479993923547
#train_cols = ['LotFrontageNorm','GrLivArea']
train_cols = ['LotFrontage','GrLivArea','OverallQualCond']  #0.5972533775871418

In [ ]:
### Fit a linear regression model
model_lr = LinearRegression(normalize=False)
model_lr.fit(X_train[train_cols], np.log(y_train))


In [ ]:
## score the model
train_score_lm=model_lr.score(X_train[train_cols], np.log(y_train))
print('lm training score     : ', train_score_lm)


In [ ]:
hold_out_score_lm=model_lr.score(X_hold_out[train_cols], np.log(y_hold_out))
print('lm test score         : ', hold_out_score_lm)

In [ ]:
#score = the coefficient of determination R^2 of the prediction 
#The coefficient R^2 is defined as (1 - u/v), 
#  where u is the residual sum of squares ((y_true - y_pred) ** 2).sum() 
#  and v is the total sum of squares ((y_true - y_true.mean()) ** 2).sum(). 
#  The best possible score is 1.0 and it can be negative (because the model can be arbitrarily worse)

In [ ]:
# run the training data through the predict to get the residuals
predicted = np.exp(pd.DataFrame(model_lr.predict(X_train[train_cols])))
expected = y_train

In [ ]:
predicted.head()

In [ ]:
expected.head()

In [ ]:
if save_intermediate_work:
    predicted.to_csv('excluded/log-test-predicted-train-lr.csv',index=False)
    pd.DataFrame(expected).to_csv('excluded/log-test-expected-train.csv',index=False)

In [ ]:
# plot the predicted vs the expected.  
# how well did the model perform
plt.plot([0,800000],[0,800000], color='red')
plt.title('Visualize the predicictions performance')
plt.xlabel('predicted')
plt.ylabel('expected/observed')
plt.scatter(predicted, expected)

In [ ]:
residuals = np.array(expected.values.ravel()) - np.array(predicted.values.ravel())
plt.title('residuals')
plt.xlabel('expected/observed')
plt.ylabel('residuals')
plt.plot([0,800000],[0,0], color='red')
plt.scatter(expected, residuals)
# a good residual plot does not have a pattern
# a pattern in the plot means we missed in capturing the data thats driving sale price

In [ ]:
# do a preditction on the hold out data
predicted = np.exp(pd.DataFrame(model_lr.predict(X_hold_out[train_cols])))
predicted.head()

In [ ]:
expected = y_hold_out
expected.head()

In [ ]:
if save_intermediate_work:
    predicted.to_csv('excluded/log-test-predicted-hold-out-lr.csv',index=False)
    pd.DataFrame(expected).to_csv('excluded/log-test-expected-hold-out.csv',index=False)

In [ ]:
# plot the predicted vs the expected.  
# how well did the model perform
plt.plot([0,800000],[0,800000], color='red')
plt.title('Visualize the predicictions performance')
plt.xlabel('predicted')
plt.ylabel('expected/observed')
plt.scatter(predicted, expected)

In [ ]:
errors = np.array(expected.values.ravel()) - np.array(predicted.values.ravel())
plt.title('errors')
plt.xlabel('expected/observed')
plt.ylabel('errors')
plt.plot([0,800000],[0,0], color='red')
plt.scatter(expected, errors)
# a good residual plot does not have a pattern
# a pattern in the plot means we missed in capturing the data thats driving sale price

In [ ]:
y_train.head()

In [ ]:
# Fit a random forest
model_rf = RandomForestRegressor(random_state=9261774, n_estimators=200, min_samples_split=2, min_samples_leaf=1, max_features='sqrt', max_depth=None, bootstrap=False)
model_rf.fit(X_train[train_cols], y_train)


In [ ]:
train_score_rf=model_rf.score(X_train[train_cols], y_train)
train_score_rf

In [ ]:
X_hold_out[train_cols].head()

In [ ]:
hold_out_score_rf=model_rf.score(X_hold_out[train_cols], y_hold_out)
hold_out_score_rf

In [ ]:
# run the training data through the predict to get the residuals
predicted = pd.DataFrame(model_rf.predict(X_train[train_cols]))
predicted.head()

In [ ]:
expected = y_train
expected.head()

In [ ]:
if save_intermediate_work:
    predicted.to_csv('excluded/log-test-predicted-train-rf.csv',index=False)

In [ ]:
# plot the predicted vs the expected.  
# how well did the model perform
plt.plot([0,800000],[0,800000], color='red')
plt.title('Visualize the predicictions performance')
plt.xlabel('predicted')
plt.ylabel('expected/observed')
plt.scatter(predicted, expected)

In [ ]:
residuals = np.array(expected.values.ravel()) - np.array(predicted.values.ravel())
plt.title('residuals')
plt.xlabel('expected/observed')
plt.ylabel('residuals')
plt.plot([0,800000],[0,0], color='red')
plt.scatter(expected, residuals)
# a good residual plot does not have a pattern
# a pattern in the plot means we missed in capturing the data thats driving sale price

In [ ]:
# do a preditction on the hold out data
predicted = pd.DataFrame(model_rf.predict(X_hold_out[train_cols]))
# put 
expected = y_hold_out

In [ ]:
# plot the predicted vs the expected.  
# how well did the model perform
plt.plot([0,800000],[0,800000], color='red')
plt.title('Visualize the predicictions performance')
plt.xlabel('predicted')
plt.ylabel('expected/observed')
plt.scatter(predicted, expected)

In [ ]:
errors = np.array(expected.values.ravel()) - np.array(predicted.values.ravel())
plt.title('errors')
plt.xlabel('expected/observed')
plt.ylabel('errors')
plt.plot([0,800000],[0,0], color='red')
plt.scatter(expected, errors)
# a good residual plot does not have a pattern
# a pattern in the plot means we missed in capturing the data thats driving sale price

In [ ]:
###
### proceed past this point when ready to make a prediction
###

In [ ]:
X_submission[train_cols].describe()

In [ ]:
# for the vars we are about to use in the predict cal, are there any NaN(s)?
X_submission[train_cols].isna().any()


In [ ]:
### lr predict 
lr_submission_df = X_submission[['Id']]
lr_prediction_y=model_lr.predict(X_submission[train_cols])
# tack the saved predictions onto the preds into a data frame
lr_pred_df=pd.DataFrame(lr_prediction_y, columns=['SalePrice'])
lr_submission_df = pd.concat([lr_submission_df,lr_pred_df], axis='columns', sort=False)


In [ ]:
lr_submission_df.shape

In [ ]:
lr_submission_df.head()

In [ ]:
### save submission file
lr_submission_df.to_csv('excluded/_log-test-submission-lr.csv',index=False)

In [ ]:
### rf predict 
rf_submission_df = X_submission[['Id']]
rf_prediction_y=model_rf.predict(X_submission[train_cols])
# tack the saved predictions onto the preds into a data frame
rf_pred_df=pd.DataFrame(rf_prediction_y, columns=['SalePrice'])
rf_submission_df = pd.concat([rf_submission_df,rf_pred_df], axis='columns', sort=False)


In [ ]:
rf_submission_df.shape

In [ ]:
rf_submission_df.head()

In [ ]:
### save submission file
rf_submission_df.to_csv('excluded/_log-test-submission-rf.csv',index=False)


In [ ]:
lr_rf_submission_df = pd.concat([lr_submission_df,rf_pred_df], axis='columns', sort=False)

In [ ]:
lr_rf_submission_df.columns=['Id', 'SalePrice_LR','SalePrice_RF']

lr_rf_submission_df.head()

In [ ]:
lr_rf_submission_df['SalePrice'] = (lr_rf_submission_df['SalePrice_LR'] * 0.50) + (lr_rf_submission_df['SalePrice_RF'] * 0.50)
lr_rf_submission_df.head()

In [ ]:
lr_rf_submission_df.drop('SalePrice_LR', axis=1, inplace=True)
lr_rf_submission_df.drop('SalePrice_RF', axis=1, inplace=True)
lr_rf_submission_df.head()

In [ ]:
lr_rf_submission_df.to_csv('excluded/_log-test-submission-lr-rf.csv',index=False)

In [ ]:
# the above code received a kaggle score of 
#
#   0.28700  train_cols = ['LotFrontage','GrLivArea'] - lr
#
#   0.24117  train_cols = ['LotFrontage','GrLivArea','OverallQualCond'] - lr
#   0.22596  train_cols = ['LotFrontage','GrLivArea','OverallQualCond'] - rf
#   0.21537  train_cols = ['LotFrontage','GrLivArea','OverallQualCond'] - ensemble lr+rf
#


# More data analysis needed
# Practically every independent variable in this dataset is useful
# there are a few variablex that have excessive NaN(s), probably best to drop the variables
# Create combined columns, ex total bathroom count, total overall condition score
# Separate the "categorical" type columns into numeric categoricals and one hot encoded variables
# create additional models and ensemble them together
#    model_rf = RandomForestRegressor(random_state=9261774, n_estimators=400, ...
#    model_gb = GradientBoostingRegressor(random_state=9261774,learning_rate=0.025,...
#    ...
#    submission_lr_rf_gb['SalePrice']=(
#        submission_lr_rf_gb['SalePrice_LR'] * 0.20 +
#        submission_lr_rf_gb['SalePrice_RF'] * 0.30 +
#        submission_lr_rf_gb['SalePrice_GB'] * 0.50 )
# 
# my best score, with a few days of data cleaning/feature engineering
#
# 0.12593
# 1744/5749